In [2]:
import os

In [3]:
os.environ['MLFLOW_TRACKING_URI'] ="https://dagshub.com/erenknowstocode/data_science_project.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']="erenknowstocode"
os.environ['MLFLOW_TRACKING_PASSWORD']="02662f30438c4c9f1058932f8ddb8a8fd588daaf"

In [4]:
os.chdir('../')

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metrics_file_name: Path
    target_column: str
    mlflow_uri: str

In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import *

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_file=CONFIG_FILE_PATH,
                 params_file=PARAMS_FILE_PATH,
                 schema_file=SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_file)
        self.params = read_yaml(params_file)
        self.schema = read_yaml(schema_file)
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])
        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params=params,
            metrics_file_name=config.metrics_file_name,
            target_column=schema.name,
            mlflow_uri= "https://dagshub.com/erenknowstocode/data_science_project.mlflow"
        )
        return model_evaluation_config

In [8]:
import os
import pandas as pd
import joblib
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn 

In [9]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config
    
    def evaluation_metrics(self,actual,pred):
        rmse = np.sqrt(mean_absolute_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        
        return rmse, mae, r2
    
    def log_in_to_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[self.config.target_column]
        
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)
            
            (rmse, mae, r2) = self.evaluation_metrics(test_y,predicted_qualities)
            
            scores = {"rmse":rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metrics_file_name),data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)
            
            if tracking_uri_type_store != "file":
                mlflow.sklearn.log_model(model,"model",registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model,"model")

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.log_in_to_mlflow()
except Exception as err:
    raise err

[2025-05-26 23:17:38,376]: INFO: common: yaml file: config/config.yaml loaded successfully.]
[2025-05-26 23:17:38,379]: INFO: common: yaml file: params.yaml loaded successfully.]
[2025-05-26 23:17:38,380]: INFO: common: yaml file: schema.yaml loaded successfully.]
[2025-05-26 23:17:38,380]: INFO: common: Created directory at: artifacts]
[2025-05-26 23:17:38,381]: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-05-26 23:17:39,209]: INFO: common: Json file saved at: artifacts/model_evaluation/metrics.json]


2025/05/26 23:17:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2025/05/26 23:17:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run agreeable-quail-823 at: https://dagshub.com/erenknowstocode/data_science_project.mlflow/#/experiments/0/runs/dfdf4e27b91f4bc1ba699f08ad25a300
🧪 View experiment at: https://dagshub.com/erenknowstocode/data_science_project.mlflow/#/experiments/0
